In [1]:
import keras
import numpy as np
from pond.tensor import NativeTensor, PrivateEncodedTensor, PublicEncodedTensor

/home/koen/anaconda2/envs/openmined/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Average Pooling Layer with NativeTensor
Some small unit-tests for the AVG pooling layer. run forward pass with mnist image, and backward pass with random noise

In [2]:
class AveragePooling2D():

    def __init__(self, pool_size, strides=None):
        """ Average Pooling layer
            pool_size: (n x m) tuple
            strides: int with stride size
            Example: AveragePooling2D(pool_size=(2,2))
        """
        self.pool_size = pool_size
        self.pool_area = pool_size[0] * pool_size[1]
        self.cache = None
        self.initializer = None
        if strides == None:
            self.strides = pool_size[0]
        else:
            self.strides = strides

    def initialize(self):
        pass

    def forward(self,x):
        s = (x.shape[1] - self.pool_size[0]) // self.strides + 1
        self.initializer = type(x)
        pooled = self.initializer(np.zeros((x.shape[0], s, s, x.shape[3])))
        for j in range(s):
            for i in range(s):
                pooled[:, j, i, :] = x[:, j * self.strides:j * self.strides + self.pool_size[0], i * self.strides:i * self.strides + self.pool_size[1], :].sum(axis=(1, 2))
        
        pooled = pooled / self.pool_area
        self.cache = x
        return pooled
    
    def backward(self, d_y, learning_rate):
        x = self.cache
        d_y_expanded = d_y.repeat(self.pool_size[0], axis=1)
        d_y_expanded = d_y_expanded.repeat(self.pool_size[1], axis=2)
        d_x = d_y_expanded * x / self.pool_area
        return d_x      
    

In [3]:
# read data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
image_batch = x_train[0:64,:,:,np.newaxis] / 255.0

## NativeTensor

In [4]:
# forward pass
avg_pooling_layer = AveragePooling2D(pool_size=(2,2))
output = avg_pooling_layer.forward(NativeTensor(image_batch))
# backward pass (with random update)
delta = NativeTensor(np.random.normal(loc=1, scale=100, size=output.shape))
lr = 0.01
_ = avg_pooling_layer.backward(d_y=delta, learning_rate=lr)

## PublicEncodedTensor

In [5]:
# forward pass
avg_pooling_layer = AveragePooling2D(pool_size=(2,2))
output = avg_pooling_layer.forward(PublicEncodedTensor(image_batch))
# backward pass (with random update)
delta = PublicEncodedTensor(np.random.normal(loc=1, scale=100, size=output.shape))
lr = 0.01
_ = avg_pooling_layer.backward(d_y=delta, learning_rate=lr)

## PrivateEncodedTensor

In [6]:
# forward pass
avg_pooling_layer = AveragePooling2D(pool_size=(2,2))
output = avg_pooling_layer.forward(PrivateEncodedTensor(image_batch))
# backward pass (with random update)
delta = PrivateEncodedTensor(np.random.normal(loc=1, scale=100, size=output.shape))
lr = 0.01
_ = avg_pooling_layer.backward(d_y=delta, learning_rate=lr)